In [1]:
'''
# Dowload and install MeCab
pip install pandas
# trouble shooting with `numpy` version 
pip install numpy==2.0.0
pip install mecab-python
pip install unidic-lite
'''
import re
import os
import collections
import pandas as pd
from pprint import pprint

from termextract import LRValue
# from termextract.preprocessing import extract_nouns, morphemes_to_surface, MeCabTokenizer
from termextract.preprocessing import MeCabTokenizer

import word_wakati as wkt

In [2]:
df = pd.read_csv("datasets/bachelor_themes.csv",encoding='UTF-8')
target_labs = [
    '朝廣研','安部研','石田健研','合志研','下川研',
    '成研','田中研','仲研','米元研','アプドゥハン研',
    '稲永研','于研','石田俊研','澤田研','安武研','前田研']

target_docs_df = df[df["研究室"].isin(target_labs )] 
target_docs_df.head() 

,年度,学科／研究科,研究室,学籍番号,賞,氏名,要旨,題目
3,H17,情報科学科,成研,NaN,NaN,NaN,NaN,対面教育を支援するためのウェブ助言システム
4,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,衛星通信検討のための気象データの解析-2002年のデータ解析-
6,H17,情報科学科,仲研,NaN,NaN,NaN,NaN,生化学反応シミュレータの比較評価
7,H17,情報科学科,澤田研,NaN,NaN,NaN,NaN,FPGAを搭載したロボット教材のためのIPとしてKUE-CHIP2の検討
9,H17,情報科学科,石田健研,NaN,NaN,NaN,NaN,重畳積文法によるX線CTの画像再構成-Shepp-Loganのフィルタ関数の適用-


In [3]:
years = df["年度"].unique()
years

array(['H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24', 'H25',
       'H26', 'H27', 'H28', 'H29', 'H30', 'R01', 'R02', 'R03', 'R04'],
      dtype=object)

In [4]:
docs = {}
for y in years:
    for b in target_labs:
        titles = target_docs_df[(target_docs_df['年度']==y) & (target_docs_df['研究室']==b)]['題目'].values.tolist()
        # print(f"{y}:{b}")
        docs[(y, b)] = titles

r04_ishida = docs['R04', '石田健研']
print('。'.join(r04_ishida))

MathematicaとMathTeXを用いた線形代数の教材作成 ―4次の行列式、クラーメルの公式―。MathematicaとMathTeXを用いた微分積分の教材作成 ―合成関数の微分と有理数べき関数の追加―。プログラミング環境による連立一次方程式求解の速度比較。最小点の探索におけるアルゴリズムの調査 ―探索アルゴリズムによる実行時間の比較―。層状媒質の1次元FDTD法解析 ―パルスの反射波から層状媒質の比誘電率分布の推定―。マイクロ波を用いる誘電体円柱の誘電率推定に対するMATLABニューラルネットワークの検証。九産大12号館建物内に侵入したローカル5Gの電波測定


In [7]:
text ='。'.join(r04_ishida)
tok = MeCabTokenizer() 
compound_nouns = []
for sentence in text.splitlines():
    tokens = tok.iter_token(sentence)
    nouns = tok.extract_nouns(tokens)
    print(nouns)
    if len(nouns) > 0 :
        compound_nouns.extend(nouns)

lr = LRValue()
lr_values = lr.fit_transform(compound_nouns)
lr_values = [(k,v) for k,v in lr_values.items() if v>0]
data_collection = sorted(lr_values, key=lambda x:x[1], reverse=True)
list(data_collection[:20])

[['Mathematica'], ['MathTeX'], ['線形', '代数'], ['教材', '作成'], ['4', '次'], ['行列', '式'], ['クラーメル'], ['公式'], ['Mathematica'], ['MathTeX'], ['微分', '積分'], ['教材', '作成'], ['合成', '関数'], ['微分'], ['有理', '数'], ['関数'], ['追加'], ['プログラミング', '環境'], ['連立', '一', '次', '方程', '式', '求', '解'], ['速度', '比較'], ['小点'], ['探索'], ['アルゴリズム'], ['調査'], ['探索', 'アルゴリズム'], ['実行', '時間'], ['比較'], ['層状', '媒質'], ['1', '次元', 'FDTD', '法', '解析'], ['パルス'], ['反射', '波'], ['層状', '媒質'], ['比', '誘電', '率', '分布'], ['推定'], ['マイクロ', '波'], ['誘電', '体', '円柱'], ['誘電', '率', '推定'], ['MATLAB', 'ニューラル', 'ネットワーク'], ['検証'], ['九', '産', '大', '12', '号', '館', '建物', '内'], ['侵入'], ['ローカル', '5', 'G'], ['電波', '測定']]


[('教材 作成', np.float64(3.4641016151377544)),
 ('層状 媒質', np.float64(3.4641016151377544)),
 ('誘電 率 推定', np.float64(2.2894284851066637)),
 ('比 誘電 率 分布', np.float64(2.0296635898134046)),
 ('Mathematica', np.float64(2.0)),
 ('MathTeX', np.float64(2.0)),
 ('誘電 体 円柱', np.float64(2.0)),
 ('連立 一 次 方程 式 求 解', np.float64(1.919471219577405)),
 ('4 次', np.float64(1.8612097182041991)),
 ('行列 式', np.float64(1.8612097182041991)),
 ('九 産 大 12 号 館 建物 内', np.float64(1.8340080864093424)),
 ('1 次元 FDTD 法 解析', np.float64(1.7411011265922482)),
 ('MATLAB ニューラル ネットワーク', np.float64(1.5874010519681994)),
 ('ローカル 5 G', np.float64(1.5874010519681994)),
 ('反射 波', np.float64(1.5650845800732873)),
 ('マイクロ 波', np.float64(1.5650845800732873)),
 ('線形 代数', np.float64(1.4142135623730951)),
 ('微分 積分', np.float64(1.4142135623730951)),
 ('合成 関数', np.float64(1.4142135623730951)),
 ('微分', np.float64(1.4142135623730951))]